In [19]:
!pip install -U chromadb langchain langchain-groq langchain-community langchain-chroma langchain-text-splitters sentence-transformers unstructured "unstructured[pdf]"


In [35]:
import os
from langchain.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [21]:
# pip install --upgrade unstrectured[local-inference]

In [22]:
# one code is missing here

In [ ]:
os.environ["GROQ_API_KEY"]="your_api_key_here"

In [24]:
# Fetch the PDF from the URL
import requests
url = "https://dspmuranchi.ac.in/pdf/Blog/Python%20Built-In%20Functions.pdf"
response = requests.get(url)

In [25]:
response

<Response [200]>

In [26]:
# Save the PDF to a local file
with open("python_inbuildfunction.pdf", "wb") as f:
    f.write(response.content)

In [27]:
# Save the PDF to a local file
with open("python_inbuildfunction.pdf","wb") as f:
    f.write(response.content)

In [29]:
text_splitter = CharacterTextSplitter(chunk_size = 100, chunk_overlap = 50)

In [30]:
text_splitter

In [31]:
texts = text_splitter.split_documents(documents)

In [32]:
type(texts)

list

In [33]:
texts[10]

Document(metadata={'source': 'python_inbuildfunction.pdf'}, page_content=">>> ascii('ușor')\n\n“‘u\\\\u0219or'”\n\nLet’s apply it to a list.\n\n>>> ascii(['s','ș'])")

In [36]:
HuggingFaceEmbeddings()

/tmp/ipython-input-3540458490.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings()
/tmp/ipython-input-3540458490.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### HuggingFaceEmbeddings

The `HuggingFaceEmbeddings` class in LangChain allows you to use models from the Hugging Face library to generate embeddings for your text data. Embeddings are numerical representations of text that capture semantic meaning, which are useful for tasks like similarity search and retrieval.

When you initialize `HuggingFaceEmbeddings()`, it downloads a default sentence transformer model (often `sentence-transformers/all-mpnet-base-v2`). You can also specify a different model by passing its name to the constructor, like this:

In [37]:
presist_directory = "vector_db"

In [43]:
vectordb=Chroma.from_documents(texts, embeddings, persist_directory=presist_directory)

In [40]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [44]:
print(vectordb)

In [45]:
retriever = vectordb.as_retriever()

In [46]:
llm = ChatGroq(model = "llama-3.3-70b-versatile", temperature=0)

In [47]:
 qa_chain= RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [48]:
query='explin about filter, map,reduce'
response=qa_chain.invoke({"query":query})

In [49]:
print(response)

{'query': 'explin about filter, map,reduce', 'result': "Here's an explanation of `filter`, `map`, and `reduce`:\n\n### Filter\n\n`filter()` is a built-in Python function that takes two arguments: a function and an iterable (such as a list, tuple, or set). It applies the function to each item in the iterable and returns a new iterable that includes only the items for which the function returns `True`.\n\nThe general syntax of `filter()` is:\n```python\nfilter(function, iterable)\n```\nHere's an example:\n```python\ndef is_even(num):\n    return num % 2 == 0\n\nnumbers = [1, 2, 3, 4, 5, 6]\neven_numbers = list(filter(is_even, numbers))\nprint(even_numbers)  # [2, 4, 6]\n```\nIn this example, the `is_even` function is applied to each item in the `numbers` list, and the resulting iterable includes only the even numbers.\n\n### Map\n\n`map()` is another built-in Python function that applies a function to each item in an iterable and returns a new iterable with the results.\n\nThe general sy

In [50]:
print(response['result'])

Here's an explanation of `filter`, `map`, and `reduce`:

### Filter

`filter()` is a built-in Python function that takes two arguments: a function and an iterable (such as a list, tuple, or set). It applies the function to each item in the iterable and returns a new iterable that includes only the items for which the function returns `True`.

The general syntax of `filter()` is:
```python
filter(function, iterable)
```
Here's an example:
```python
def is_even(num):
    return num % 2 == 0

numbers = [1, 2, 3, 4, 5, 6]
even_numbers = list(filter(is_even, numbers))
print(even_numbers)  # [2, 4, 6]
```
In this example, the `is_even` function is applied to each item in the `numbers` list, and the resulting iterable includes only the even numbers.

### Map

`map()` is another built-in Python function that applies a function to each item in an iterable and returns a new iterable with the results.

The general syntax of `map()` is:
```python
map(function, iterable)
```
Here's an example:
```p